<a href="https://colab.research.google.com/github/hwyum/deeplearning_nlp/blob/master/gluon_rnn_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [0]:
import os
os.chdir("drive/My Drive/Colab Notebooks/DLC_NLP lecture/")

# Gluon RNN 예제코드 (Time Machine)

In [0]:
# ## data loading을 위한 패키지 설치
# !pip install gutenberg

In [0]:
# !git clone https://github.com/c-w/Gutenberg.git

In [0]:
# !apt-get install libdb++-dev
# !export BERKELEYDB_DIR=/usr
# !pip install -r ./Gutenberg/requirements-py3.pip

## package import

In [6]:
!apt-get update
!apt-get upgrade
!pip install mxnet-cu92mkl
!pip install gluonnlp matplotlib tqdm

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1710/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1710/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  Release
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [83.2 kB]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [519 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [711 kB]
Get:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic/main 

In [0]:
import pandas as pd
import numpy as np
# from konlpy.tag import Mecab
from mxnet.gluon import nn, rnn
from mxnet import gluon, autograd
import gluonnlp as nlp
from mxnet import nd 
import mxnet as mx
import multiprocessing as mp
from multiprocessing import Pool
import time
import itertools
from tqdm import tqdm, tqdm_notebook, trange

mx.random.seed(1)
ctx = mx.gpu()

## 데이터 로딩

In [0]:
with open("./data/timemachine.txt") as f:
  time_machine = f.read()

In [9]:
print(time_machine[0:500])


The Project Gutenberg EBook of The Time Machine, by H. G. Wells

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.net


Title: The Time Machine

Author: H. G. Wells

Release Date: October 2, 2004 [EBook #35]
Last Updated: January 14, 2018

Language: English

Character set encoding: UTF-8

*** START OF 


In [10]:
len(time_machine)

199135

In [11]:
print(time_machine[180000:180244])

story. And I have by me, for my comfort, two strange white
flowers—shrivelled now, and brown and flat and brittle—to witness that
even when mind and strength had gone, gratitude and a mutual tenderness
still lived on in the heart of man.









In [0]:
time_machine = time_machine[:180244]

In [0]:
# \n을 ' '로 대체

time_machine = time_machine.replace('\n','')


## 데이터 전처리

### Numerical representations of characters


*   create numerical representations of characters
*   use one-hot representations
*   need to have definitive list of characters



In [14]:
character_list = list(set(time_machine))
vocab_size = len(character_list)
print(character_list)
print('length of vocab: %s' % vocab_size)

['u', ':', 'C', 'S', 'v', '5', ']', '2', 'f', 'y', 'p', 'X', ')', '”', 'r', 'D', '(', 'M', '…', '[', 'g', 'm', 'E', 'W', ';', 'o', 'w', 'R', 'ç', 'n', 'H', '3', 'c', 'æ', 'Y', ' ', '.', 'l', 'F', '*', 'i', '-', 'N', 'J', 'T', 'a', 'O', 'd', '’', '“', 's', 'P', 'z', 't', 'b', '4', '1', '?', '—', 'L', '‘', '0', 'k', 'j', '#', 'e', 'G', '!', 'Q', 'U', ',', 'A', 'V', 'œ', 'K', 'I', 'x', '_', 'ü', 'B', 'q', '8', 'h']
length of vocab: 83


In [15]:
character_dict = {}
for e, char in enumerate(character_list):
  character_dict[char] = e
print(character_dict)

{'u': 0, ':': 1, 'C': 2, 'S': 3, 'v': 4, '5': 5, ']': 6, '2': 7, 'f': 8, 'y': 9, 'p': 10, 'X': 11, ')': 12, '”': 13, 'r': 14, 'D': 15, '(': 16, 'M': 17, '…': 18, '[': 19, 'g': 20, 'm': 21, 'E': 22, 'W': 23, ';': 24, 'o': 25, 'w': 26, 'R': 27, 'ç': 28, 'n': 29, 'H': 30, '3': 31, 'c': 32, 'æ': 33, 'Y': 34, ' ': 35, '.': 36, 'l': 37, 'F': 38, '*': 39, 'i': 40, '-': 41, 'N': 42, 'J': 43, 'T': 44, 'a': 45, 'O': 46, 'd': 47, '’': 48, '“': 49, 's': 50, 'P': 51, 'z': 52, 't': 53, 'b': 54, '4': 55, '1': 56, '?': 57, '—': 58, 'L': 59, '‘': 60, '0': 61, 'k': 62, 'j': 63, '#': 64, 'e': 65, 'G': 66, '!': 67, 'Q': 68, 'U': 69, ',': 70, 'A': 71, 'V': 72, 'œ': 73, 'K': 74, 'I': 75, 'x': 76, '_': 77, 'ü': 78, 'B': 79, 'q': 80, '8': 81, 'h': 82}


In [0]:
time_numerical = [character_dict[char] for char in time_machine]

In [17]:
# check format
print(len(time_numerical))
print(time_numerical[:20])

#convert back to text
print("".join([character_list[idx] for idx in time_numerical[:50]]))

177032
[44, 82, 65, 35, 51, 14, 25, 63, 65, 32, 53, 35, 66, 0, 53, 65, 29, 54, 65, 14]
The Project Gutenberg EBook of The Time Machine, b


### One-hot representations
We can use NDArray’s one_hot() operation to render a one-hot representation of each character. But frack it, since this is the from scratch tutorial, let’s write this ourselves.



In [0]:
def one_hots(numerical_list, vocab_size=vocab_size):
  result = nd.zeros((len(numerical_list), vocab_size), ctx=ctx)
  for i, idx in enumerate(numerical_list):
    result[i, idx] = 1.0
  return result

In [19]:
time_numerical[:2]

[44, 82]

In [20]:
print(one_hots(time_numerical[:2]))


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
<NDArray 2x83 @gpu(0)>


In [0]:
def textify(embedding):
  result = ""
  indices = nd.argmax(embedding, axis=1).asnumpy()
  for idx in indices:
    result += character_list[int(idx)]
  return result

In [22]:
textify(one_hots(time_numerical[:40]))

'The Project Gutenberg EBook of The Time '

### Preparing the data for training

In [23]:
(len(time_numerical)-1 // 64) * 64
# len(time_numerical)
# time_numerical[]

11330048

In [24]:
## make sequence
seq_length = 64

# -1 here so we have enough characters for labels later
num_samples = (len(time_numerical)-1) // seq_length
dataset = one_hots(time_numerical[:seq_length*num_samples]).reshape((num_samples,seq_length, vocab_size))
textify(dataset[0])

'The Project Gutenberg EBook of The Time Machine, by H. G. WellsT'

#### preparing input batch 

In [25]:
dataset.shape

(2766, 64, 83)

In [0]:
batch_size = 32

In [27]:
print('# of sequences in dataset: ', len(dataset))
num_batches = len(dataset) // batch_size
print('# of batches: ', num_batches)
train_data = dataset[:num_batches*batch_size].reshape((batch_size, num_batches, seq_length, vocab_size))


# of sequences in dataset:  2766
# of batches:  86


In [28]:
train_data.shape #(batch_size, num_batches, seq_length, vocab_size)

(32, 86, 64, 83)

In [29]:
# swap batch_size and seq_length axis to make later access easier
train_data = nd.swapaxes(train_data, 0, 1)
train_data = nd.swapaxes(train_data, 1, 2)
print('Shape of data set:' , train_data.shape)

Shape of data set: (86, 64, 32, 83)


In [30]:
for i in range(3):
    print("***Batch %s:***\n %s \n %s \n\n" % (i, textify(train_data[i, :, 0]), textify(train_data[i, :, 1])))


***Batch 0:***
 The Project Gutenberg EBook of The Time Machine, by H. G. WellsT 
 Four-Dimensioned being, which is a fixed andunalterable thing.“S 


***Batch 1:***
 his eBook is for the use of anyone anywhere at no cost and witha 
 cientific people,” proceeded the Time Traveller, after the pause 


***Batch 2:***
 lmost no restrictions whatsoever.  You may copy it, give it away 
 required for the proper assimilation of this, “know very well th 




#### Preparing our labels

In [31]:
labels = one_hots(time_numerical[1:seq_length*num_samples+1])
train_label = labels.reshape((batch_size, num_batches, seq_length, vocab_size))
train_label = nd.swapaxes(train_label, 0, 1)
train_label = nd.swapaxes(train_label, 1, 2)
print(train_label.shape)


(86, 64, 32, 83)


### A Final Sanity Check
Remember that our target at every time step is to predict the next character in the sequence. So our labels should look just like our inputs but offset by one character. Let’s look at corresponding inputs and outputs to make sure everything lined up as expected.

In [32]:
print(textify(train_data[10, :, 3]))
print(textify(train_label[10, :, 3]))


 a larger edition of the littlemechanism which we had seen vanis
a larger edition of the littlemechanism which we had seen vanish


## Recurrent neural networks

Recall that the update for an ordinary hidden layer in a neural network with activation function ϕ is given by

*  h=ϕ(xW+b)

in RNN, need to add a weight sum of the previous hidden state h(t-1)
*  ht=ϕ( x(t)W(xh)+h(t)−1W(hh)+b(h) )

Then at every time set t we'll calculate the output as:
y(t)_hat = softmax(h(t)W(hy)+b(y)


### Allocate parameters

In [0]:
num_inputs = vocab_size
num_hidden = 256
num_outputs = vocab_size

########################
#  Weights connecting the inputs to the hidden layer
########################
Wxh = nd.random_normal(shape=(num_inputs,num_hidden), ctx=ctx) * .01

########################
#  Recurrent weights connecting the hidden layer across time steps
########################
Whh = nd.random_normal(shape=(num_hidden,num_hidden), ctx=ctx) * .01

########################
#  Bias vector for hidden layer
########################
bh = nd.random_normal(shape=num_hidden, ctx=ctx) * .01


########################
# Weights to the output nodes
########################
Why = nd.random_normal(shape=(num_hidden,num_outputs), ctx=ctx) * .01
by = nd.random_normal(shape=num_outputs, ctx=ctx) * .01

# NOTE: to keep notation consistent,
# we should really use capital letters
# for hidden layers and outputs,
# since we are doing batchwise computations]

### Attach the gradients

In [0]:
params = [Wxh, Whh, bh, Why, by]
for param in params:
  param.attach_grad()

### Softmax Activation

In [0]:
def softmax(y_linear, temperature=1.0):
  lin = (y_linear-(nd.max(y_linear, axis=1).reshape((-1,1)))) / temperature
  exp = nd.exp(lin)
  partition = nd.sum(exp, axis=1).reshape((-1,1))
  return exp / partition

In [36]:
####################
# With a temperature of 1 (always 1 during training), we get back some set of probabilities
####################
softmax(nd.array([[1, -1], [-1, 1]]), temperature=1.0)



[[0.880797   0.11920292]
 [0.11920292 0.880797  ]]
<NDArray 2x2 @cpu(0)>

In [37]:
####################
# Often we want to sample with low temperatures to produce sharp probabilities
####################
softmax(nd.array([[10,-10],[-10,10]]), temperature=0.1)



[[1. 0.]
 [0. 1.]]
<NDArray 2x2 @cpu(0)>

In [38]:
####################
# If we set a high temperature, we can get more entropic (*noisier*) probabilities
####################
softmax(nd.array([[1,-1],[-1,1]]), temperature=1000.0)



[[0.50049996 0.49949998]
 [0.49949998 0.50049996]]
<NDArray 2x2 @cpu(0)>

### Define the model

In [0]:
def simple_rnn(inputs, state, temperature=1.0):
  outputs = []
  h = state
  for X in inputs:
    h_linear = nd.dot(X, Wxh) + nd.dot(h, Whh) + bh
    h = nd.tanh(h_linear)
    yhat_linear = nd.dot(h, Why) + by
    yhat = softmax(yhat_linear, temperature = temperature)
    outputs.append(yhat)
  return(outputs, h)

### Cross-entropy loss function

In [0]:
# def cross_entropy(yhat, y):
#     return - nd.sum(y * nd.log(yhat))

def cross_entropy(yhat, y):
    return - nd.mean(nd.sum(y * nd.log(yhat), axis=0, exclude=True))

In [41]:
cross_entropy(nd.array([[.2,.5,.3], [.2,.5,.3]]), nd.array([[1.,0,0], [0, 1.,0]]))



[1.1512926]
<NDArray 1 @cpu(0)>

### Aeraging the loss over the sequence
Because the unfolded RNN has multiple outputs (one at every time step) we can calculate a loss at every time step. The weights corresponding to the net at time step t influence both the loss at time step t and the loss at time step t+1. To combine our losses into a single global loss, we’ll take the average of the losses at each time step.

In [0]:
def average_ce_loss(outputs, labels):
  assert(len(outputs) == len(labels))
  total_loss = 0
  for(output, label) in zip(outputs, labels):
    total_loss = total_loss + cross_entropy(output,label)
  return total_loss / len(outputs)
  

### Optimizer

In [0]:
def SGD(params, lr):
  for param in params:
    param[:] = param - lr * param.grad

## Generating text by sampling

We have now defined a model that takes a sequence of real inputs from our training data and tries to predict the next character at every time step. You might wonder, what can we do with this model? Why should I care about predicting the next character in a sequence of text?

This capability is exciting because given such a model, we can now generate strings of plausible text. The generation procedure goes as follows. 


---

Say our string begins with the character “T”. We can feed the letter “T” and get a conditional probability distribution over the next character P(x2|x1="T"). We can the sample from this distribution, e.g. producing an “i”, and then assign x2="i", feeding this to the network at the next time step.



In [0]:
def sample(prefix, num_chars, temperature=1.0):
    #####################################
    # Initialize the string that we'll return to the supplied prefix
    #####################################
    string = prefix

    #####################################
    # Prepare the prefix as a sequence of one-hots for ingestion by RNN
    #####################################
    prefix_numerical = [character_dict[char] for char in prefix]
    input_sequence = one_hots(prefix_numerical)

    #####################################
    # Set the initial state of the hidden representation ($h_0$) to the zero vector
    #####################################
    sample_state = nd.zeros(shape=(1, num_hidden), ctx=ctx)

    #####################################
    # For num_chars iterations,
    #     1) feed in the current input
    #     2) sample next character from from output distribution
    #     3) add sampled character to the decoded string
    #     4) prepare the sampled character as a one_hot (to be the next input)
    #####################################
    for i in range(num_chars):
        outputs, sample_state = simple_rnn(input_sequence, sample_state, temperature=temperature)
        choice = np.random.choice(vocab_size, p=outputs[-1][0].asnumpy())
        string += character_list[choice]
        input_sequence = one_hots([choice])
    return string

In [0]:
epochs = 2000
moving_loss = 0.

learning_rate = .5

# state = nd.zeros(shape=(batch_size, num_hidden), ctx=ctx)
for e in range(epochs):
    ############################
    # Attenuate the learning rate by a factor of 2 every 100 epochs.
    ############################
    if ((e+1) % 100 == 0):
        learning_rate = learning_rate / 2.0
    state = nd.zeros(shape=(batch_size, num_hidden), ctx=ctx)
    for i in range(num_batches):
        data_one_hot = train_data[i]
        label_one_hot = train_label[i]
        with autograd.record():
            outputs, state = simple_rnn(data_one_hot, state)
            loss = average_ce_loss(outputs, label_one_hot)
            loss.backward()
        SGD(params, learning_rate)

        ##########################
        #  Keep a moving average of the losses
        ##########################
        if (i == 0) and (e == 0):
            moving_loss = np.mean(loss.asnumpy()[0])
        else:
            moving_loss = .99 * moving_loss + .01 * np.mean(loss.asnumpy()[0])

    print("Epoch %s. Loss: %s" % (e, moving_loss))
    print(sample("The Time Ma", 1024, temperature=.1))
    print(sample("The Medical Man rose, came to the lamp,", 1024, temperature=.1))


Epoch 0. Loss: 3.768621200973361
The Time Maeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeneeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee